In [3]:
import tensorflow as tf
import numpy as np
from sklearn import datasets
import pandas as pd
import seaborn as sns

np.random.seed(0)
%matplotlib inline

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))


def model(X, w):
    return tf.matmul(X, w)

In [4]:
# generate skwed data (the ratio of class 1 to class 0 is defined by weights)
X, y = datasets.make_classification(n_samples=30000, n_features=10,
                                    n_informative=2, n_redundant=2,
                                    weights = [0.7]
                                    )
# use 60% of the data for training
train_ratio = 0.6
train_samples = int(len(y) * train_ratio)

X_train = X[:train_samples]
X_test = X[train_samples:]
y_train = y[:train_samples]
y_test = y[train_samples:]

# Create a DataFrame about few statistics of the data generated for only illustration purpose  
counts = pd.DataFrame(columns=["nrows", "ncols", "nrows_C0", "nrows_C1", "C0 ratio", "C1 ratio"],
                      index=["data", "training", "testing"])
counts.loc["data",] = [X.shape[0], X.shape[1], 
                       sum(y==0), sum(y==1), 
                       1.*sum(y==0)/len(y), 1.*sum(y==1)/len(y)]
counts.loc["training",] = [X_train.shape[0], X_train.shape[1], 
                           sum(y_train==0), sum(y_train==1), 
                           1.*sum(y_train==0)/len(y_train), 1.*sum(y_train==1)/len(y_train)]
counts.loc["testing",] = [X_test.shape[0], X_test.shape[1], 
                          sum(y_test==0), sum(y_test==1), 
                          1.*sum(y_test==0)/len(y_test), 1.*sum(y_test==1)/len(y_test)]

counts

,nrows,ncols,nrows_C0,nrows_C1,C0 ratio,C1 ratio
data,30000,10,20956,9044,0.6985333,0.3014667
training,18000,10,12545,5455,0.6969444,0.3030556
testing,12000,10,8411,3589,0.7009167,0.2990833


In [9]:
ncol = X_train.shape[1]
X = tf.placeholder("float", [None, ncol]) # create symbolic variables
Y = tf.placeholder("float")

w = init_weights([ncol, 1])

py_x = model(X, w)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y)) 

train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost) 
predict_op = tf.argmax(py_x, 1) 

sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

for i in range(100):
    for start, end in zip(range(0, len(X_train), 128), range(128, len(X_train), 128)):
        sess.run(train_op, feed_dict={X: X_train[start:end], Y: y_train[start:end]})
    print i, np.mean(np.argmax(y_test, axis=1) ==
                     sess.run(predict_op, feed_dict={X: X_test, Y: y_test}))
    

InvalidArgumentError: logits and labels must be same size: logits_size=dim { size: 128 } dim { size: 1 } labels_size=dim { size: 128 }
	 [[Node: SoftmaxCrossEntropyWithLogits_3 = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](MatMul_3, _recv_Placeholder_9_0)]]
Caused by op u'SoftmaxCrossEntropyWithLogits_3', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/valar/venv/base/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/valar/venv/base/local/lib/python2.7/site-packages/traitlets/config/application.py", line 592, in launch_instance
    app.start()
  File "/home/valar/venv/base/local/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 389, in start
    ioloop.IOLoop.instance().start()
  File "/home